In [1]:
import os
import dill

import numpy as np

import TwoPUtils
import STX3KO_analyses as stx



%matplotlib inline

%load_ext autoreload
%autoreload 2

/home/mplitt/anaconda3/envs/STX3_test/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [8]:
def update_sess_dict(mouse, f, KO = True):
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
#     if KO:
#         try:
#             source_folder  =os.path.join('/media','mplitt','Backup Plus','2P_Data')
#             source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
#             info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
#         except:
#             source_folder  = os.path.join('/media','mplitt','Backup Plus2','2P_Data')
#             source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
#             info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
#     else:
#         try:
#             source_folder  =os.path.join('/media','mplitt','Backup Plus3')
#             source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
#             info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
#         except:
#             source_folder  = os.path.join('/media','mplitt','Backup Plus5','2P_Data')
#             source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
#             info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')

#     f['mouse']=mouse
#     f.update({'scan_file': source_stem + '.sbx',
#               'scanheader_file': source_stem + '.mat',
#               'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
#               'scan_number': f['scan'],
#               'prompt_for_keys': False,
#               'VR_only': False,
#               'scanner': "NLW",
#              })
    
#     source_folder = "/mnt/BigDisk/2P_scratch"
    source_folder = '/media/mplitt/Backup Plus2/'
    source_stem = os.path.join(source_folder, mouse, f['date'], f['scene'], scan_str)
    print(f['scene'], f['session'], f['scan'])
    f['mouse']=mouse
    f.update({'scan_file': source_stem + '.sbx',
              'scanheader_file': source_stem + '.mat',
              'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
              's2p_path': os.path.join("/mnt/BigDisk/2P_scratch", f['mouse'], f['date'], f['scene'],
                                         "%s_%.03d_%.03d" % (f['scene'], f['session'], f['scan']), "suite2p"),
              'scan_number': f['scan'],
              'prompt_for_keys': False,
              'VR_only': False,
              'scanner': "NLW",
             })
    return f
    

In [3]:
f = stx.morph_sess_deets.CTRL_sessions['4467331.1'][0]
print(type(f['scene']), type(f['session']), type(f['scan']))
"%s_%.03d" % (f['scene'], f['session'])

<class 'str'> <class 'int'> <class 'int'>


'TwoTower_foraging_002'

In [9]:
def run_and_save(fdict,comb_frames=0):
#     with open(os.path.join('/home/mplitt/YMazeSessPkls',mouse,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
#         sess = dill.load(file)
#     sess.novel_arm = fdict['novel']    
            
    sess = stx.session.MorphSession(**fdict)
    sess.load_scan_info()
    sess.align_VR_to_2P()
    print(fdict['s2p_path'])
    sess.load_suite2p_data(which_ts=('F', 'Fneu','spks'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
    sess.get_trial_info()
    sess.neuropil_corrected_dff()
    
    
    sess.place_cells_calc(nperms=1000)
    
    sess.add_pos_binned_trial_matrix('F_dff')
    sess.add_timeseries(F_dff_norm = sess.timeseries['F_dff']/np.nanpercentile(sess.timeseries['F_dff'],99,axis=1,keepdims=True))
    sess.rm_pos_binned_trial_matrix('F_dff_norm')
    sess.add_pos_binned_trial_matrix('F_dff_norm')
    
    sess.add_pos_binned_trial_matrix('spks') #,speed=sess.vr_data['dz']._values, speed_thr = .05)  
    sess.add_timeseries(spks_norm = sess.timeseries['spks']/np.nanpercentile(sess.timeseries['spks'],99,axis=1,keepdims=True))
    sess.add_pos_binned_trial_matrix('spks_norm') 

    sess.add_timeseries(licks=sess.vr_data['lick']._values)
    sess.add_pos_binned_trial_matrix('licks')
    sess.add_timeseries(speed=sess.vr_data['dz']._values)
    sess.add_pos_binned_trial_matrix('speed')
    
    TwoPUtils.sess.save_session(sess,'/home/mplitt/MorphSessPkls')
    return comb_frames+sess.scan_info['max_idx']

In [10]:

for mouse, sessions in stx.morph_sess_deets.KO_sessions.items():
    for f in sessions:
        print(f)

    
        if isinstance(f,dict):
            f_ = update_sess_dict(mouse,f)
    #         print(f_)
            _ = run_and_save(f_)
        elif isinstance(f, tuple) or isinstance(f, list):
            comb_frames = 0

            for _f in f:
                print(f)

    #             _f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",mouse,f[0]['date'],'combined','suite2p')
                f_ = update_sess_dict(mouse,_f)
                print(f_)
                comb_frames = run_and_save(f_)
        
        
        
    

{'date': '09_10_2020', 'scene': 'TwoTower_foraging', 'session': 4, 'scan': 11, 'mouse': '4467975.1', 'scan_file': '/mnt/BigDisk/2P_scratch/4467975.1/09_10_2020/TwoTower_foraging/TwoTower_foraging_004_011.sbx', 'scanheader_file': '/mnt/BigDisk/2P_scratch/4467975.1/09_10_2020/TwoTower_foraging/TwoTower_foraging_004_011.mat', 'vr_filename': '/home/mplitt/VR_scratch/4467975.1/09_10_2020/TwoTower_foraging_4.sqlite', 'scan_number': 11, 'prompt_for_keys': False, 'VR_only': False, 'scanner': 'NLW'}
Fixing teleports
(71626,) (29091,)
1881.6069984840829 1881.5159171298637
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
({'date': '11_10_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 4, 'mouse': '4467975.1', 'scan_file': '/mnt/BigDisk/2P_scratch/4467975.1/11_10_2020/TwoTower_foraging/TwoTower_foraging_002_004.sbx', 'scanheader_file': '/mnt/BigDisk/2P_scratch/4467975.1/11_10_2020/TwoTower_foraging/TwoTower_foraging_002_004.mat', 'vr_filename': '/home/mplitt/VR_scratch/4467975.1/11_10_2020/TwoTower_foraging_2.sqlite', 'scan_number': 4, 'prompt_for_keys': False, 'VR_only': False, 'scanner': 'NLW'}, {'date': '11_10_2020', 'scene': 'TwoTower_foraging', 'session': 4, 'scan': 10})
{'date': '11_10_2020', 'scene': 'TwoTower_foraging', 'session': 4, 'scan

/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '13_10_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 5}
Fixing teleports
(63394,) (23406,)
1513.9262253663467 1513.8150581101568
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


...

perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '14_10_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 3}
Fixing teleports
(77183,) (28825,)
1864.4288782213239 1864.3112683173322
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '15_10_2020', 'scene': 'TwoTower_foraging', 'ses

/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '13_10_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 5}
Fixing teleports
(67111,) (25003,)
1617.210965133906 1617.1076301162204
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '14_10_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 4}
Fixing teleports
(99391,) (36843,)
2382.9729661445176 2382.908539666498
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '15_10_2020', 'scene': 'TwoTower_foraging', 'sess

In [10]:
for mouse, sessions in stx.morph_sess_deets.CTRL_sessions.items():
    print(mouse)
    for f in sessions:
        print(f)
        if isinstance(f,dict):
            f_ = update_sess_dict(mouse,f)
            print(f_['s2p_path'])
            _ = run_and_save(f_)
        elif isinstance(f, tuple) or isinstance(f, list):
            comb_frames = 0
            for _f in f:
                print(f)
                f_ = update_sess_dict(mouse,_f)
                print(f_)
                comb_frames = run_and_save(f_)
        

4467331.1
{'date': '10_12_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 1, 'mouse': '4467331.1', 'scan_file': '/media/mplitt/Backup Plus2/4467331.1/10_12_2020/TwoTower_foraging/TwoTower_foraging_002_001.sbx', 'scanheader_file': '/media/mplitt/Backup Plus2/4467331.1/10_12_2020/TwoTower_foraging/TwoTower_foraging_002_001.mat', 'vr_filename': '/home/mplitt/VR_scratch/4467331.1/10_12_2020/TwoTower_foraging_2.sqlite', 's2p_path': '/home/mplitt/2P_scratch/4467331.1/10_12_2020/TwoTower_foraging/TwoTower_foraging_002_001/suite2p', 'scan_number': 1, 'prompt_for_keys': False, 'VR_only': False, 'scanner': 'NLW'}
TwoTower_foraging 2 1
/mnt/BigDisk/2P_scratch/4467331.1/10_12_2020/TwoTower_foraging/TwoTower_foraging_002_001/suite2p
Fixing teleports
(86996,) (26003,)
1681.9339312784234 1681.7867609903992
/mnt/BigDisk/2P_scratch/4467331.1/10_12_2020/TwoTower_foraging/TwoTower_foraging_002_001/suite2p
/mnt/BigDisk/2P_scratch/4467331.1/10_12_2020/TwoTower_foraging/TwoTower_foraging_002_001/

/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '15_12_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 2}
TwoTower_foraging 2 2
/mnt/BigDisk/2P_scratch/4467331.1/15_12_2020/TwoTower_foraging/TwoTower_foraging_002_002/suite2p
Fixing teleports
(65644,) (23406,)
1513.9200353713998 1513.8150581101568
/mnt/BigDisk/2P_scratch/4467331.1/15_12_2020/TwoTower_foraging/TwoTower_foraging_002_002/suite2p
/mnt/BigDisk/2P_scratch/4467331.1/15_12_2020/TwoTower_foraging/TwoTower_foraging_002_002/suite2p
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '16_12_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 4}
TwoTower_foraging 2 4
/mnt/BigDisk/2P_scratch/4467331.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_002_004/suite2p
Fixing teleports
(76845,) (27463,)
1776.3477766548763 1776.2182920667005
/mnt/BigDisk/2P_scratch/4467331.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_002_004/suite2p
/mnt/BigDisk/2P_scratch/4467331.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_002_004/suite2p
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
p

...

perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '17_12_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 2}
TwoTower_foraging 2 2
/mnt/BigDisk/2P_scratch/4467332.1/17_12_2020/TwoTower_foraging/TwoTower_foraging_002_002/suite2p
Fixing teleports
(68498,) (24830,)
1606.012127337039 1605.9181404749875
/mnt/BigDisk/2P_scratch/4467332.1/17_12_2020/TwoTower_foraging/TwoTower_foraging_002_002/suite2p
/mnt/BigDisk/2P_scratch/4467332.1/17_12_2020/TwoTower_foraging/TwoTower_foraging_002_002/suite2p
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
pe

/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '16_12_2020', 'scene': 'TwoTower_foraging', 'session': 3, 'scan': 1}
TwoTower_foraging 3 1
/mnt/BigDisk/2P_scratch/4467333.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_003_001/suite2p
Fixing teleports
(56153,) (19828,)
1282.4796614451743 1282.3931278423447
/mnt/BigDisk/2P_scratch/4467333.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_003_001/suite2p
/mnt/BigDisk/2P_scratch/4467333.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_003_001/suite2p
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:144: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:155: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '17_12_2020', 'scene': 'TwoTower_foraging', 'session': 3, 'scan': 4}
TwoTower_foraging 3 4
/mnt/BigDisk/2P_scratch/4467333.1/17_12_2020/TwoTower_foraging/TwoTower_foraging_003_004/suite2p
Fixing teleports
(73486,) (27140,)
1755.4324153612936 1755.3269327943408
/mnt/BigDisk/2P_scratch/4467333.1/17_12_2020/TwoTower_foraging/TwoTower_foraging_003_004/suite2p
/mnt/BigDisk/2P_scratch/4467333.1/17_12_2020/TwoTower_foraging/TwoTower_foraging_003_004/suite2p
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
p

In [10]:
KO_failed_sessions = run_all_sessions(stx.morph_sess_deets.KO_sessions)

/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")


Fixing teleports
(71626,) (29091,)
1881.6069984840829 1881.5159171298637


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")


Fixing teleports
(32153,) (13716,)
887.1888580090956 887.0742799393634


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")


...

Fixing teleports
(66550,) (24872,)
1608.7638959070239 1608.634663971703


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")


Fixing teleports
(71094,) (26702,)
1727.0656897422941 1726.9974734714503


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")


Fixing teleports
(93560,) (34961,)
2261.2602324406266 2261.182415361294


In [6]:
sess.vr_data

NameError: name 'sess' is not defined